Building and training a multitask neural network with GlassPy
=============================================================



In [ ]:
!pip install glasspy

## Introduction



In this notebook we will create and train a multitask neural network. This is a neural network that can predict more than one target. The procedure is very similar to what we did in the previous notebook.



## Imports



In [ ]:
import torch
import lightning as L
import numpy as np
from sklearn.model_selection import train_test_split
from glasspy.data import SciGlass
from sklearn.preprocessing import MaxAbsScaler
from torch.utils.data import DataLoader, TensorDataset
from glasspy.predict.base import MTL

## Data pipeline



Again we need to collect, process and split the data, similar to the previous notebook. This time we will create a model to predict the density and refractive index.



### Collecting the data



In [ ]:
config_prop = {
    "must_have_or": ["Density293K", "RefractiveIndex"],
}

config_comp = {
    "must_have_only": [
        "SiO2",
        "Li2O",
        "Na2O",
        "K2O",
        "CaO",
        "MgO",
        "BaO",
        "Al2O3",
        "TiO2",
    ],
}

source = SciGlass(
    elements_cfg={},
    properties_cfg=config_prop,
    compounds_cfg=config_comp,
)

source.remove_duplicate_composition(
    scope="compounds",
    decimals=3,
    aggregator="median",
)

df = source.data

df["property"].info()

In [ ]:
idx = df.index

X = df["compounds"]
y = df["property"][["Density293K", "RefractiveIndex"]]

### Splitting the data



In [ ]:
TEST_SIZE = 0.1
RANDOM_SEED = 61455

In [ ]:
indices = df.index
train_val_idx, test_idx = train_test_split(
    indices, test_size=TEST_SIZE, random_state=RANDOM_SEED
)

X_train_val = X.loc[train_val_idx]
y_train_val = y.loc[train_val_idx]

X_test = X.loc[test_idx].values
y_test = y.loc[test_idx].values

In [ ]:
train_idx, val_idx = train_test_split(
    train_val_idx, test_size=TEST_SIZE, random_state=RANDOM_SEED
)

X_train = X.loc[train_idx].values
y_train = y.loc[train_idx].values

X_val = X.loc[val_idx].values
y_val = y.loc[val_idx].values

### Normalization



In [ ]:
x_scaler = MaxAbsScaler()
x_scaler.fit(X_train)

y_scaler = MaxAbsScaler()
y_scaler.fit(y_train)

X_train = x_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)

X_val = x_scaler.transform(X_val)
y_val = y_scaler.transform(y_val)

X_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### The DataModule class



In [ ]:
class DataModule(L.LightningDataModule):
    def __init__(
        self,
        X_train,
        y_train,
        X_val,
        y_val,
        X_test,
        y_test,
        x_scaler=None,
        y_scaler=None,
        batch_size = 256,
        num_workers = 2,
    ):
        super().__init__()

        self.batch_size = batch_size
        self.num_workers = num_workers

        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test
        self.x_scaler = x_scaler
        self.y_scaler = y_scaler


    def train_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_train, self.y_train),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_val, self.y_val),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_test, self.y_test),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

In [ ]:
dm = DataModule(
    X_train, y_train, X_val, y_val, X_test, y_test, x_scaler, y_scaler
)

## Neural network



In [ ]:
NUM_EPOCHS = 20
num_features = X.shape[1]
num_targets = y.shape[1]

hparams = {
    "batch_size": 256,
    "layer_1_activation": "Sigmoid",
    "layer_1_size": 3,
    "layer_2_activation": "Sigmoid",
    "layer_2_size": 2,
    "loss": "mse",
    "max_epochs": NUM_EPOCHS,
    "n_features": num_features,
    "n_targets": num_targets,
    "num_layers": 2,
    "optimizer": "SGD",
}

In [ ]:
my_mtl = MTL(hparams)

### Training the neural network



In [ ]:
trainer = L.Trainer(max_epochs=NUM_EPOCHS)

In [ ]:
trainer.fit(my_mtl, dm)

### Testing the neural network



In [ ]:
dm.setup("test")

X_true = dm.X_test

y_true = dm.y_test
y_true = dm.y_scaler.inverse_transform(y_true)

y_pred = my_mtl.predict(X_true)
y_pred = dm.y_scaler.inverse_transform(y_pred)

RMSE_density, RMSE_refractiveindex = my_mtl.RMSE(y_true, y_pred)

print(RMSE_density)
print(RMSE_refractiveindex)

### Saving the model



In [ ]:
STATE_DICT = "model.pth"
LEARNING_CURVE = "learning_curve.p"
HPARAMS = "hparams.p"

In [ ]:
my_mtl.save_training(STATE_DICT, LEARNING_CURVE, HPARAMS)

In [ ]:
loaded_model = MTL.from_file(HPARAMS, STATE_DICT, LEARNING_CURVE)

In [ ]:
dm.setup("test")

X_true = dm.X_test

y_true = dm.y_test
y_true = dm.y_scaler.inverse_transform(y_true)

y_pred = loaded_model.predict(X_true)
y_pred = dm.y_scaler.inverse_transform(y_pred)

RMSE_density, RMSE_refractiveindex = loaded_model.RMSE(y_true, y_pred)

print(RMSE_density)
print(RMSE_refractiveindex)